In [ ]:
import anndata as ad
import h5py
import json
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.neural_network import MLPClassifier
from statistics import mode
import tensorflow as tf
from tqdm import trange

from functions import *

%load_ext autoreload
%autoreload 2
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 30})

folder_bin = folder = DATA_PATH
folder_saved = f'{folder}/art_saved/'

# Load and prepare data  

In [ ]:
NUM_CLASSES = 8 #select the NUM_CLASSES largest classes

In [ ]:
metadata = pd.read_csv(f'{folder}cell_metadata.csv')
f = h5py.File(f'{folder}C57BL6J-638850-raw.h5ad', 'r')
cell_id = f['obs']['cell_label'][:]
cell_id = [cell_id[i].decode() for i in range(len(cell_id))]
brain_section = f['obs']['brain_section_label']['codes'][:]
version = '20231215'
f = open(f'{folder}manifest.json')
manifest = json.load(f)
print("version: ", manifest['version'])
file = f'{folder}cluster_to_cluster_annotation_membership_pivoted.csv'
cluster_details = pd.read_csv(file,keep_default_na=False)
cluster_details.set_index('cluster_alias', inplace=True)
file = f'{folder}/cluster_to_cluster_annotation_membership_color.csv'
cluster_colors = pd.read_csv(file,keep_default_na=False)
cluster_colors.set_index('cluster_alias', inplace=True)
metadata_extended = metadata.join(cluster_details,on='cluster_alias')
metadata_extended = metadata_extended.join(cluster_colors,on='cluster_alias')
heights = np.sort(np.array(list(dict.fromkeys(metadata_extended.z))))
file = f'{folder}gene.csv.1'
gene = pd.read_csv(file)
gene.set_index('gene_identifier',inplace=True)
print("Number of genes = ", len(gene))
adata = ad.read_h5ad(f'{folder}C57BL6J-638850-raw.h5ad', backed='r')
adatalog = ad.read_h5ad(f'{folder}C57BL6J-638850-log2.h5ad', backed='r')
print('------\nDONE: data successfully loaded\n-----')
section_names  = list(dict.fromkeys(metadata.brain_section_label.values))
pred = [x in section_names for x in metadata_extended['brain_section_label']]
sections = metadata_extended[pred] # this is (num_cells)x21
print(np.array(pred).sum())
gnames = gene['gene_symbol'].values[:100]
pred = [x in gnames for x in adata.var.gene_symbol]
gene_filtered = adata.var[pred]
asubset = adata[:,gene_filtered.index].to_memory() #this is 4mln x (num_genes)
asubsetlog = adatalog[:,gene_filtered.index].to_memory() #this is 4mln x (num_genes)
gdata = asubset.to_df()
gdatalog = asubsetlog.to_df()
gdata.columns = gnames
gdatalog.columns = gnames
joined = sections.join(gdata, 'cell_label')
joinedlog = sections.join(gdatalog, 'cell_label')
data = joined[gnames].to_numpy()
datalog = joinedlog[gnames].to_numpy()
print('----------\n DATA SUCCESSFULLY SELECTED\n----------')
# volumes = np.nanmean(data/(2**datalog-1),1) ### only for fraction
del data
del datalog
section_names  = list(dict.fromkeys(metadata.brain_section_label.values))
pred = [x in section_names for x in metadata_extended['brain_section_label']]
sections = metadata_extended[pred] # this is (num_cells)x21
print(np.array(pred).sum())
gnames = gene['gene_symbol'].values[:510]
pred = [x in gnames for x in adata.var.gene_symbol]
gene_filtered = adata.var[pred]
asubset = adata[:,gene_filtered.index].to_memory() #this is 4mln x (num_genes)
gdata = asubset.to_df()
gdata.columns = gnames
joined = sections.join(gdata, 'cell_label')
# -------------------
data = joined[gnames].to_numpy()
print('----------\n DATA SUCCESSFULLY SELECTED\n----------')

In [ ]:
sublcasses = sections['subclass'].values
vals,counts= np.unique(sublcasses, return_counts=True)
classes = sections['class'].values
listclasses = list(dict.fromkeys(classes))
del metadata
del metadata_extended
del gene_filtered
del gdata
del sections
del pred


labels = np.loadtxt(f'{folder}labels.dat') #all dataset label
# ------------------------------------------------------------------
classes = labels
listclasses = np.sort(list(dict.fromkeys(labels)))
sizes = np.array([(labels==i).sum() for i in listclasses])
xi = np.vstack([data[classes==listclasses[i]][:,:500] for i in np.argsort(sizes)[::-1][:NUM_CLASSES]])
ordered_listclasses = listclasses[np.argsort(sizes)[::-1]]
label = np.hstack([[ordered_listclasses[i]]*np.sort(sizes)[::-1][i] for i in range(NUM_CLASSES)])
label_list = np.array(list(dict.fromkeys(label))).astype(int)
#------------------
data_classes = np.vstack(xi)
subcl =  np.hstack([sublcasses[classes==listclasses[i]] for i in np.argsort(sizes)[::-1][:NUM_CLASSES]])
v,c = np.unique(subcl, return_counts=True)
with open(f'{folder}SUBCLASSES.dat', 'w') as f:
    for line in subcl:
        f.write(f"{line}\n")

# Statistics of raw counts (fig1-sum_cc.pdf)

In [ ]:
## distribution of raw counts
x = data_classes
bins = np.arange(x.min(),x.max()+2,)-0.5
HISTO= []
for i in trange(500):
    histo = np.histogram(x[:,i],bins);
    HISTO.append(histo[0]) 
HISTO = np.array(HISTO)
# SAVE
np.savetxt(f'{folder_saved}_average_histo_rawc.dat',HISTO.mean(0))

In [ ]:
## distribution of raw correlations
cc = np.corrcoef(data_classes.T)
# SAVE
np.savetxt(f'{folder_saved}cc_raw.dat', cc[np.triu_indices(500,1)]) 

In [ ]:
## distribution of raw sum (and comparison with suffled case)
data=x
# ----- shuffle data
datas =np.copy(data)
all_idx = []
for i in trange(len(datas.T)):
    np.random.shuffle(datas[:,i]) 
hh1 = np.histogram(data.sum(1),-0.5+np.arange(data.sum(1).min(), data.sum(1).max()+2) );
hh2 = np.histogram(datas.sum(1),-0.5+np.arange(datas.sum(1).min(), datas.sum(1).max()+2));
# SAVE
np.savetxt(f'{folder_saved}histo_sumraw.dat', np.vstack((hh1[1][1:],hh1[0]/hh1[0].sum())))
np.savetxt(f'{folder_saved}histo_sumraw_shuffled.dat', np.vstack((hh2[1][1:],hh2[0]/hh2[0].sum())))

# Classification accuracy single raw count (acc_1gene.pdf)

In [ ]:
data = np.loadtxt(f'{folder}data.dat')
labels = np.loadtxt(f'{folder}labels.dat')

# -------------------- load binary data
NUM_CLASSES = 8
classes = labels
listclasses = np.sort(list(dict.fromkeys(labels)))
sizes = np.array([(labels==i).sum() for i in listclasses])
xi = np.vstack([data[classes==listclasses[i]][:,:500] for i in np.argsort(sizes)[::-1][:NUM_CLASSES]])
ordered_listclasses = listclasses[np.argsort(sizes)[::-1]]
label = np.hstack([[ordered_listclasses[i]]*np.sort(sizes)[::-1][i] for i in range(NUM_CLASSES)])
label_list = np.array(list(dict.fromkeys(label))).astype(int)
label = np.hstack([[ordered_listclasses[i]]*np.sort(sizes)[::-1][i] for i in range(NUM_CLASSES)])
# --------------------- distribution of classes
pclass=[]
for k in label_list:
    pclass.append((label ==k).sum()/len(label))
pclass = np.array(pclass)
np.savetxt(f'{folder_saved}pclass.dat', pclass)

### -------------------  plot and save
# plt.plot(np.sort(sizes)[::-1], 'o', color='black')
# plt.yscale('log')
# plt.axhline(500+500*499/2, color= 'grey', ls='--')
# plt.xlabel('i')
# plt.ylabel('size class i')
# np.savetxt(f'{folder_saved}allclass_size.dat', sizes)

In [ ]:
ax = np.arange(len(xi))
y = label

ax_shuffled = np.random.choice(ax,len(xi), replace=False)
train_size = int(0.8*len(ax_shuffled))

xi_train = xi[ax_shuffled][:train_size]
xi_test =  xi[ax_shuffled][train_size:]
y_train =  y[ax_shuffled][:train_size]
y_test =   y[ax_shuffled][train_size:] 

d=1
saved=[]

for kk in trange(0,500):
    random_genes=[kk]
    x_train = xi_train[:,random_genes]
    x_test =  xi_test[:,random_genes]
    hindi =[]
    for k in label_list:
        zi = x_train[y_train==k]  
        mu = zi.mean(0)
        h = -np.arctanh(mu-1e-15)
        hindi.append(h)
    hindi = np.array(hindi)

    prob_xgiveny = (np.exp(-hindi[:,None,:]*x_test[None])/(np.exp(-hindi)+np.exp(hindi))[:,None,:]).prod(-1)
    prob_ygivenx = prob_xgiveny*pclass[:,None]
    prob_ygivenx = prob_ygivenx/prob_ygivenx.sum(0)
    y_pred_condind = label_list[np.argmax(prob_ygivenx,0)]
    acc = (y_pred_condind == y_test).sum()/len(y_test)
    saved.append((kk,acc))

svd = np.array(saved)
ordered_variables= (svd[:,0][np.argsort(svd[:,1])[::-1]]).astype(int)
svd_all = np.vstack((svd, [0,(y_test==1).sum()/len(y_test)]))

In [ ]:
## Select groups of genes to use at each size d (vary d)
d = 50
# -----------------------------------------------------
saved =[]
for boundary_selection in [d+50, -d-50, 500]:
    for kk in trange(50):
        random_genes=np.random.choice(ordered_variables[:boundary_selection], d, replace=False)
        x_train = xi_train[:,random_genes]
        x_test =  xi_test[:,random_genes]
        hindi =[]
        for k in label_list:
            zi = x_train[y_train==k]  
            mu = zi.mean(0)
            h = -np.arctanh(mu-1e-15)
            hindi.append(h)
        hindi = np.array(hindi)
        prob_xgiveny = (np.exp(-hindi[:,None,:]*x_test[None])/(np.exp(-hindi)+np.exp(hindi))[:,None,:]).prod(-1)
        prob_ygivenx = prob_xgiveny*pclass[:,None]
        prob_ygivenx = prob_ygivenx/prob_ygivenx.sum(0)
        y_pred_condind = label_list[np.argmax(prob_ygivenx,0)]
        acc = (y_pred_condind == y_test).sum()/len(y_test)
        saved.append(np.hstack((random_genes, acc)))
        
svd = np.array(saved)
plt.plot((svd[:,-1]),'o')
plt.show()
random_genes = svd[np.argsort(svd[:,-1])[::-1]][0][:-1].astype(int)
np.savetxt(f'{folder}_genes_{d}.dat',random_genes)
print(random_genes)

In [ ]:
# Accuracy of raw data

acc_nat =[]
for d in [3,5,10,15,30,50,100,200,500]:
    random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
    #----------------------------
    x_train = x_train_all[:,random_genes]
    x_test =  x_test_all[:,random_genes]
    # ----------------
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(), 
                        random_state=1, batch_size=64, learning_rate_init =1e-4,
                       verbose=True, max_iter=1000, tol=1e-3)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy = (y_pred==y_test).sum()/len(y_pred)
    #--------------
    print(d, accuracy)  
    acc_nat.append((d, accuracy))    
    np.savetxt(f'{folder_saved}acc_nat.dat', np.array(acc_nat))

In [ ]:
# Use a NN to find the accuracy

acc_n =[]
for d in [2,3,5,10,30,50,100,200,500]:
    random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
    #----------------------------
    x_train = x_train_all[:,random_genes]
    x_test =  x_test_all[:,random_genes]
    # ----------------
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100), 
                        random_state=1, batch_size=64, learning_rate_init =1e-4,
                       verbose=True, max_iter=1000, tol=1e-3)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy = (y_pred==y_test).sum()/len(y_pred)
    #--------------
    print(d, accuracy)  
    acc_n.append((d, accuracy))    
    np.savetxt(f'{folder_saved}acc_n.dat', np.array(acc_n))

In [ ]:
# Average accuracy with binary data
x = xi
y = label
#----------------------------
ax = np.arange(len(x))
ax_shuffled = np.random.choice(ax,500000, replace=False)
train_size = int(0.8*len(ax_shuffled))    
#----------------------------
x_train_all = x[ax_shuffled][:train_size]
x_test_all =  x[ax_shuffled][train_size:]
y_train = y[ax_shuffled][:train_size]
y_test =  y[ax_shuffled][train_size:]

filepath = f'{folder_saved}acc_bin_AVG_{NUM_CLASSES}.dat'

if os.path.isfile(filepath)== False:
    acc_nat_avg = []
    np.savetxt(filepath, np.array(acc_nat_avg)) 
    print('new file generated')
acc_nat_avg = np.loadtxt(filepath)
acc_nat_avg = [i for i in acc_nat_avg]

In [ ]:
# acc_nat_avg = [i for i in acc_nat_avg]
for d in [1,2,3,5,10,15,30,50,100,200,500]:
    acc=[]
    nrepetitions=20
    if d==500: nrepetitions=1
    for _ in trange(nrepetitions):
        random_genes = np.random.choice(500,d,replace=False)
        #----------------------------
        x_train = x_train_all[:,random_genes]
        x_test =  x_test_all[:,random_genes]
        # ----------------
        clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100), 
                            random_state=1, batch_size=64, learning_rate_init =1e-3,
                           verbose=True, max_iter=1000, tol=1e-3)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        accuracy = (y_pred==y_test).sum()/len(y_pred)
        acc.append(accuracy)
        print(random_genes, acc)
        
    acc = np.array(acc)
    #--------------
    print(d, acc.mean(), acc.std())  
    acc_nat_avg.append((d, acc.mean(), acc.std()))   
    np.savetxt(filepath, np.array(acc_nat_avg))

# Statistics of binarized data (binarization.pdf)

In [ ]:
# Average of gene expression given zero probability
prob_0 = (data_classes==0).sum(0)/len(data_classes)

mean_not0 = []
for i in trange(len(data_classes.T)):
    mean_not0.append(data_classes[:,i][data_classes[:,i]!=0].mean())
mean_not0 = np.array(mean_not0)

p0_means = np.vstack((prob_0, mean_not0, data_classes.mean(0))).T
np.savetxt(f'{folder_saved}p0_means.dat', p0_means)

In [ ]:
## load binarized data
data = np.loadtxt(f'{folder}data.dat')
labels = np.loadtxt(f'{folder}labels.dat')

In [ ]:
# compare binary and raw sums
z_classes = data
cells = np.random.choice(range(len(z_classes)), 10000, replace=False)
sum_zx = np.vstack((z_classes.sum(1)[cells], data_classes.sum(1)[cells]))
np.savetxt(f'{folder_saved}sum_zx.dat', sum_zx.T) 

In [ ]:
# compute binary correlations
cc_z = np.corrcoef(z_classes.T)
np.savetxt(f'{folder_saved}cc_bin.dat', cc_z[np.triu_indices(500,1)]) 

# Convergence of Inverse Ising method (convergence_{d}_{label}.pdf)

In [ ]:
# ----------- generate f_data for the needed d
d=500
random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
z = xi[:,random_genes]
#---------------- save f_data
z1=z
mu = z1.mean(0)
mu2 = (np.cov(z1.T) + mu[:,None]*mu[None,:])[np.triu_indices(d,1)]
f_data = np.hstack((mu,mu2))
np.savetxt(f'{folder}f_data_{d}_all.dat', f_data)

### -------- save a subset for test
z = xi[:,random_genes]
random_cells = np.random.choice(len(z), 50000, replace=False)
z_selection = z[random_cells]
label_selection = label[random_cells]
np.savetxt(f'{folder}_datasel2_{d}.dat',z_selection)
np.savetxt(f'{folder}_datasel2_label_{d}.dat',label_selection)

## ------------------------  classes
for i in range(len(label_list)):
    z1 = z[label==label_list[i]]
    mu = z1.mean(0)
    mu2 = (np.cov(z1.T) + mu[:,None]*mu[None,:])[np.triu_indices(d,1)]
    f_data = np.hstack((mu,mu2))

    z1s = np.copy(z1)
    for ii in trange(len(z1s.T)):
        np.random.shuffle(z1s[:,ii])
    histo = plt.hist(z1.sum(1),np.arange(-d,d,2));
    histo2 = plt.hist(z1s.sum(1),np.arange(-d,d,2));

    np.savetxt(f'{folder}f_data_{d}_{label_list[i]}.dat', f_data)
    np.savetxt(f'{folder}_histosum_{d}_{label_list[i]}.dat',histo[0])
    np.savetxt(f'{folder}_histosums_{d}_{label_list[i]}.dat',histo2[0])

In [ ]:
### ------------------ MOMENTS
z = xi[:,random_genes]
deltadata = z-f_data[:d]
m2=[]
m3=[]
m4=[]
for _ in trange(2000):
    i,j,k,l = np.random.choice(10,4, replace=False)
    m2.append(((deltadata[:, [i,j]]).prod(1).mean(),  i,j))
    m3.append(((deltadata[:, [i,j,k]]).prod(1).mean(), i,j,k))
    m4.append(((deltadata[:, [i,j,k,l]]).prod(1).mean(),i,j,k,l))
m2=np.array(m2)
m3=np.array(m3)    
m4=np.array(m4) 
np.savetxt(f'{folder}_m2_{d}.dat',m2)
np.savetxt(f'{folder}_m3_{d}.dat',m3)
np.savetxt(f'{folder}_m4_{d}.dat',m4)

In [ ]:
### ----------------------- moments in classes
z = xi[:,random_genes]
for i in range(len(label_list)):
    z1 = z[label==label_list[i]]
    mu = z1.mean(0)
    mu2 = (np.cov(z1.T) + mu[:,None]*mu[None,:])[np.triu_indices(d,1)]
    f_data = np.hstack((mu,mu2))
    
    deltadata = z1-f_data[:d]
    m2=[]
    m3=[]
    m4=[]
    for _ in trange(2000):
        ii,j,k,l = np.random.choice(10,4, replace=False)
        m2.append(((deltadata[:, [ii,j]]).prod(1).mean(),  ii,j))
        m3.append(((deltadata[:, [ii,j,k]]).prod(1).mean(), ii,j,k))
        m4.append(((deltadata[:, [ii,j,k,l]]).prod(1).mean(),ii,j,k,l))
    m2=np.array(m2)
    m3=np.array(m3)    
    m4=np.array(m4) 
    np.savetxt(f'{folder}_m2_{d}_{label_list[i]}.dat',m2)
    np.savetxt(f'{folder}_m3_{d}_{label_list[i]}.dat',m3)
    np.savetxt(f'{folder}_m4_{d}_{label_list[i]}.dat',m4)

# Check Higher order statistics (ising_pred_{d}_{label}.pdf)

In [ ]:
## ------ save heff

d=100
label='all'

q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
zsel= np.loadtxt(f'{folder}_datasel2_500.dat')[:,random_genes]

h,J = get_hJ(q,d)
h=-h/d
J=-J/d ##this is already divided by 2

heff_all =[]
val_all =[]

for idx in trange(d):
    heff_0 = h[idx] + (J[idx]*zsel).sum(-1)
    val_0 = zsel[:,idx]
    heff_all.append(heff_0)
    val_all.append(val_0)
heff_all = np.hstack(heff_all)
val_all = np.hstack(val_all)
px = heff_all[np.argsort(heff_all)]
py = val_all[np.argsort(heff_all)]


values = np.linspace(px.min(),px.max(),20)
# ---------------------------------------------------
valuesy=[]
valuesyerr=[]
valuesx=[]
valuesxerr=[]
for i in trange(len(values)-1):
    ll=len(py[(px>values[i])*(px<values[i+1])])
    valuesx.append(px[(px>values[i])*(px<values[i+1])].mean())
    valuesy.append(py[(px>values[i])*(px<values[i+1])].mean())
    valuesyerr.append(py[(px>values[i])*(px<values[i+1])].std()/np.sqrt(ll)) 
    valuesxerr.append(px[(px>values[i])*(px<values[i+1])].std()/np.sqrt(ll)) 
valuesy = np.array(valuesy)
valuesx = np.array(valuesx)
valuesyerr=np.array(valuesyerr)
valuesxerr=np.array(valuesxerr)

tobesaved = np.vstack((valuesx, valuesy,valuesxerr,valuesyerr))
np.savetxt(f'{folder_bin}_heff_{d}_{label}.dat', tobesaved)

# Specific heat - model vs independent case (specific_heat.pdf)

See find_ising.ipynb

# Energy basins (basins_props)

In [ ]:
## save subset for energy basins (just once)
# indices = np.random.choice(range(len(data)), 500000, replace=False)
# np.savetxt(f'{folder}b_classes.dat',np.array(classes)[indices])
# np.savetxt(f'{folder}b_subclasses.dat',np.array(subclasses)[indices])
# np.savetxt(f'{folder}b_indices.dat',indices)
# np.savetxt(f'{folder}b_data.dat', x[indices].astype(int))

# ### -------- save a subset for test (just once)
# z = xi[:,random_genes]
# random_cells = np.random.choice(len(z), 50000, replace=False)
# z_selection = z[random_cells]
# label_selection = label[random_cells]
# np.savetxt(f'{folder}_datasel2_{d}.dat',z_selection)
# np.savetxt(f'{folder}_datasel2_label_{d}.dat',label_selection)

In [ ]:
# find mimima
d = 50
label='all'
q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')

zsel_all = np.loadtxt(f'{folder}_datasel2_500.dat')[:]
lsel = np.loadtxt(f'{folder}_datasel2_label_500.dat')[:]
random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
zsel = zsel_all[:,random_genes]
print(len(zsel), d)

labellist = np.array(list(dict.fromkeys(lsel)))
labellist=np.sort(labellist)
print(labellist)

if os.path.isfile(f'{folder_bin}minimaN_{d}_{label}.dat'):
    minima = list(np.loadtxt(f'{folder_bin}minimaN_{d}_{label}.dat') )
else: minima=[]
len(minima)

In [ ]:
for i in trange(len(minima), len(zsel)):
    cell= zsel[i]
    with open(f'{folder_bin}minimaN_{d}_{label}.dat','ab') as f:
        f.write(b'\n')
        np.savetxt(f, [get_minimum(cell,q)], fmt='%d', delimiter=' ', newline='')
        f.close() 

# Energy landscape (energy_landscape_{d}.png)

In [ ]:
## ----------------- save first largest minima
d=500
label='all'
minima = np.loadtxt(f'{folder_bin}minimaN_{d}_{label}.dat')
lst, size = np.unique(minima, return_counts=True,axis=0)
lst = lst[np.argsort(size)[::-1]]
size = size[np.argsort(size)[::-1]]
np.savetxt(f'{folder_bin}first_minima_{d}_{all}.dat',lst[:10] )

In [ ]:
## ----------------- energy histogram
d = 500
label='all'
q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
h,J = get_hJ(q,d)

zsel_all = np.loadtxt(f'{folder}_datasel2_500.dat')[:]
lsel = np.loadtxt(f'{folder}_datasel2_label_500.dat')[:]

random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
zsel = zsel_all[:,random_genes]
print(len(zsel), d)

basin = np.loadtxt(f'{folder_bin}basinN_{d}_{label}.dat').astype(int)

class_of_basn = np.array([mode(lsel[basin==i]) for i in range(1+int(basin.max()))])
lsel_pred = np.zeros(len(lsel))
for i in range(len(lsel_pred)):
    lsel_pred[i] = class_of_basn[basin[i]]
    
for k in [0,1,2,3,4]:
    energy_cells=[]
    for i in trange(1000):
        energy_cells.append(np.array(get_H(zsel[basin==k][i*500:(i+1)*500],J,h)))
    energy_cells = np.hstack(energy_cells)
    histo = np.histogram(energy_cells,100, density=True);
    plt.plot(histo[1][1:], histo[0])
    
    np.savetxt(f'{folder_bin}energy_basin_{k}_d{d}_{label}.dat', energy_cells)

In [ ]:
## ------------ Energy barriers
d=500
label='all'
q = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
h,J = get_hJ(q,d)
largest_minima = np.loadtxt(f'{folder_bin}first_minima_{d}_{all}.dat')
en_minima = get_H(largest_minima,J,h)

i=4
final_state=[]
final_nstep=[]
final_barrier=[]
l0=0

# final_state=list(np.loadtxt(f'{folder_bin}barriers_finalstate_{d}_{i}_.dat'))
# final_nstep= list(np.loadtxt(f'{folder_bin}barriers_final_nstep_{d}_{i}_.dat'))
# final_barrier=list(np.loadtxt(f'{folder_bin}barriers_final_barrier_{d}_{i}_.dat'))

l0 = len(final_barrier)
print(l0)

In [ ]:
for _ in range(500-l0):
    x0 = np.copy(largest_minima[i])
    x = np.copy(x0)[None,:]
    Hx = en_minima[i]
    
    numsteps=0
    hmax = Hx
    for _ in trange(10000):
        for _ in range(1):
            x, Hx = one_step_MC(x, h,J,Hx)
            numsteps+=1
            if Hx>hmax: hmax=Hx
        if (get_minimum(x[0],q) == x0).prod()!= 1:
            if len((np.where((get_minimum(x[0],q) == largest_minima).prod(1)==1)[0]>0)):
                final_state.append((np.where((get_minimum(x[0],q) == largest_minima).prod(1)==1)[0][0]))
            final_nstep.append(numsteps)
            final_barrier.append(hmax.numpy()[0])  
            
            print(len(final_state),numsteps)
            
            np.savetxt(f'{folder_bin}barriers_finalstate_{d}_{i}_.dat', np.array(final_state))
            np.savetxt(f'{folder_bin}barriers_final_nstep_{d}_{i}_.dat', np.array(final_nstep))
            np.savetxt(f'{folder_bin}barriers_final_barrier_{d}_{i}_.dat', np.array(final_barrier))
            break

# Confusion matrix (confusion_1ising.pdf and NN_vs_mixIsing.pdf)

In [ ]:
# confusion_{ds[i]}_{label}.dat'

In [ ]:
## ------------ Confusion matrix for raw accuracy NN

d=50
random_genes = np.loadtxt(f'{folder}_genes_{d}.dat').astype(int)
#----------------------------
x_train = x_train_all[:,random_genes]
x_test =  x_test_all[:,random_genes]
# ----------------
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100), 
                    random_state=1, batch_size=64, learning_rate_init =1e-2,
                   verbose=True, max_iter=10, tol=1e-3)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = (y_pred==y_test).sum()/len(y_pred)

# _____________________________________
labellist= label_list
confusion = np.zeros((len(labellist), len(labellist)))
for ik in range(len(labellist)):
    k = labellist[ik]
    for ikk in range(len(labellist)):
        kk=labellist[ikk]
        confusion[ik,ikk] = ((y_pred[y_test==k]==kk).sum()/ len(y_pred[y_test==k]))
        
np.savetxt(f'{folder}confusion_natNN_{d}.dat',confusion)

## Mix Ising Classifier

In [ ]:
def generate_ratio(x,q1,q2, NumSteps=1000,StepsTherm=10):
    d = len(x[0])
    h1,J1 = get_hJ(q1,d)
    h1 = tf.convert_to_tensor(h1,dtype=tf.float64)
    J1= tf.convert_to_tensor(J1,dtype=tf.float64)
    
    h2,J2 = get_hJ(q2,d)
    h2 = tf.convert_to_tensor(h2,dtype=tf.float64)
    J2= tf.convert_to_tensor(J2,dtype=tf.float64)

    num_part,d = x.shape
    num_par = int(d+ d*(d-1)/2)

    ratio = 0
    H2 = get_H(x,J2,h2)
    for n in trange(NumSteps):
        for _ in tf.range(StepsTherm):
            x,H2 = one_step_MC(x, h2,J2,H2)
        H1 = get_H(x,J1,h1)
        ratio_partial = tf.reduce_mean(tf.exp(-(H1-H2)))
        ratio = (n*ratio + ratio_partial)/(n+1)  
        
    return ratio.numpy()

In [ ]:
d = 50

In [ ]:
list_classes =np.array([1,30,31,33,9,29,2,28])
N_CLASSES = len(list_classes)

## ----- ---Load f_data and q for each class
label ='all'
f_data_all = np.loadtxt(f'{folder_bin}f_data_{d}_{label}.dat')
f_data_i=[]
for label in list_classes:
    f_data_i.append(np.loadtxt(f'{folder_bin}f_data_{d}_{label}.dat'))
f_data_i = np.vstack((f_data_i))
# ---------------------------------
label ='all'
q_all = np.loadtxt(f'{folder_bin}q_{d}_{label}.dat')
q_i=[]
for label in list_classes:
    q_i.append(np.loadtxt(f'{folder_bin}q_{d}_{label}.dat'))
q_i = np.vstack((q_i))

In [ ]:
#---------------------Thermalize points at q_all
q = q_all
h,J = get_hJ(q,d)
h = tf.convert_to_tensor(h,dtype=tf.float64)
J= tf.convert_to_tensor(J,dtype=tf.float64)

In [ ]:
n_samples = 8000
x = np.random.choice([-1,1], (n_samples,d))
x = tf.convert_to_tensor(x,dtype=tf.float64) 

In [ ]:
### thermalize x to the q ###
n_thermalize = 50000
x, energy_history = thermalize(x,J,h, NumSteps=n_thermalize , crange =trange)
plt.plot(energy_history)

In [ ]:
d = len(x[0])
energy_x_class =[]

for i in range(N_CLASSES):
    q = q_i[i]
    h,J = get_hJ(q,d)
    H1 = get_H(x,J,h)
    energy_x_class.append(H1)
energy_x_class = np.array(energy_x_class)

h2,J2 = get_hJ(q_all,d)
h2 = tf.convert_to_tensor(h2,dtype=tf.float64)
J2= tf.convert_to_tensor(J2,dtype=tf.float64)
H2 = get_H(x,J2,h2)

num_part,d = x.shape
num_par = int(d+ d*(d-1)/2)

In [ ]:
ratio = np.zeros(N_CLASSES)
n=0
ratio_history=[]

In [ ]:
NumSteps=1000
StepsTherm=50

for _ in trange(NumSteps):
    for _ in tf.range(StepsTherm):
        x,H2 = one_step_MC(x, h2,J2,H2)    
    energy_x_class =[]
    for i in range(N_CLASSES):
        q = q_i[i]
        h,J = get_hJ(q,d)
        H1 = get_H(x,J,h)
        energy_x_class.append(H1)
    energy_x_class = np.array(energy_x_class) 

    ratio_partial = tf.reduce_mean(tf.exp(-(energy_x_class-H2)),1)
    ratio = (n*ratio + ratio_partial)/(n+1)  
    ratio_history.append(np.array(ratio).mean())
    n+=1
ratio = ratio.numpy()

In [ ]:
plt.plot(ratio_history[:])

In [ ]:
# load Pclass from labels
pclass_ = np.loadtxt(f'{folder_bin}pclass.dat')
if (pclass_[:,0].astype(int) == list_classes).prod() ==1:
    pclass = pclass_[:,1]
    print('pclass correctly loaded')
else: 
    print('there is a problem')
    print(pclass_[:,0].astype(int), list_classes)

In [ ]:
# load x,y to test the classifier
x = np.loadtxt(f'{folder_bin}_datasel_{d}.dat')[:]
y = np.loadtxt(f'{folder_bin}_datasel_label_{d}.dat')[:]

In [ ]:
# find H(x|y)
energy_x_class =[]
for i in trange(N_CLASSES):
    q = q_i[i]
    h,J = get_hJ(q,d)  
    H1=[]
    for ii in range(int(len(x)/100)+1):
        H1.append(get_H(x[ii*100:(1+ii)*100],J,h))
    H1 = np.hstack(H1)
    energy_x_class.append(H1)
energy_x_class = np.array(energy_x_class)

In [ ]:
# find P(y|x) propto exp(-H_class)P_class/r_class
prob_y_given_x_nn = np.exp(-energy_x_class)*pclass[:,None]/ratio[:,None]
prob_y_given_x = prob_y_given_x_nn/prob_y_given_x_nn.sum(0)
np.savetxt(f'{folder_bin}pyx_{d}.txt',prob_y_given_x)

In [ ]:
prob_y_given_x = np.loadtxt(f'{folder_bin}pyx_{d}.txt')
y_pred_temp = np.argmax(prob_y_given_x,0)
y_pred = np.copy(y_pred_temp)
for i in range(N_CLASSES):
    y_pred[y_pred_temp==i]=list_classes[i]
acc = (y_pred == y).sum()/len(y)
print(d,acc)

label_list = list_classes

In [ ]:
acc_perclass =np.array([ (y_pred[y==i] == y[y==i]).sum()/len(y[y==i]) for i in label_list])
acc_perclass

In [ ]:
confusion = np.zeros((len(label_list), len(label_list)))
for ik in range(len(label_list)):
    k = label_list[ik]
    for ikk in range(len(label_list)):
        kk=label_list[ikk]
        confusion[ik,ikk] = ((y_pred[y==k]==kk).sum()/ len(y_pred[y==k]))
np.savetxt(f'{folder}confusion_mixising_{d}.dat',confusion)

# entropies_1ising.pdf

In [ ]:
sizes = [2,3,5,10,15,30,50,100,200,500]
ACC =[]
MI_all=[]
lsel = np.loadtxt(f'{folder}_datasel2_label_500.dat')
label='all'

In [ ]:
for d in sizes:
    minima = np.loadtxt(f'{folder_bin}minimaN_{d}_{label}.dat')

    lst, size = np.unique(minima, return_counts=True,axis=0)
    lst = lst[np.argsort(size)[::-1]]
    size = size[np.argsort(size)[::-1]]
    # -----
    basin =[]
    for i in trange(len(minima)):
        basin.append(np.where((minima[i] == lst).prod(1)==1)[0][0])
    basin = np.array(basin)
    
    np.savetxt(f'{folder_bin}basinN_{d}_{label}.dat',basin)
    
    class_of_basn = np.array([mode(lsel[basin==i]) for i in range(1+int(basin.max()))])
    lsel_pred = np.zeros(len(lsel))
    for i in range(len(lsel_pred)):
        lsel_pred[i] = class_of_basn[basin[i]]
    ## complessive accuracy
    accuracy = (lsel_pred == lsel).sum()/len(lsel)
    print(accuracy)
    ACC.append((d,accuracy))
    
    # --------------------
    threshold = 0.001*len(minima)
    imax = (size>threshold).sum()
    print(threshold, imax)
    labellist = np.array(list(dict.fromkeys(lsel)))
    labellist=np.sort(labellist)
    
    entropies =[]
    prob_table =[]
    for ii in trange(imax):
        prob_type_given_basin = np.array([(lsel[basin==ii]==i).sum()/len(lsel[basin==ii]) for i in labellist])
        entropy = -(prob_type_given_basin[prob_type_given_basin!=0]*np.log(prob_type_given_basin[prob_type_given_basin!=0])).sum()
        entropies.append(entropy)
        prob_table.append(prob_type_given_basin)
    entropies=np.array(entropies)
    prob_table=np.array(prob_table)
    np.savetxt(f'{folder_bin}prob_table_{d}_{label}.dat', prob_table)
    
    prob_basin = np.array([(basin==i).sum() for i in range(len(entropies))])
    prob_basin=prob_basin/prob_basin.sum()
    cond_entropy = (prob_basin*entropies).sum()
    lsel_large = lsel[basin<imax]
    prob_label = np.array([(lsel_large ==i).sum()/len(lsel_large) for i in labellist])
    entro_label = -(prob_label[prob_label!=0] * np.log(prob_label[prob_label!=0])).sum()
    MI_label_basin = entro_label - cond_entropy
    MI_all.append((d, MI_label_basin))
ACC = np.array(ACC)
MI_all = np.array(MI_all)

np.savetxt(f'{folder_bin}new_ACC_1ising_{label}.dat',ACC)
np.savetxt(f'{folder_bin}new_MI_1ising_{label}.dat',MI_all)

# Other Figures

NOTE: use find_ising.ipynb or find_ising.py to get "q_{d}_all.dat" and "q_{d}_{label}.dat" files